<a href="https://colab.research.google.com/github/open911251/open911251/blob/main/hackmd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable-baselines3[extra] sb3-contrib
!apt-get install -y python-opengl xvfb ffmpeg
!pip install gym[atari] gym[accept-rom-license] stable-baselines3[extra]
!pip install gym[atari] gym[accept-rom-license]
!pip install gym[atari] stable-baselines3[extra] --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-opengl
INFO: pip is looking at multiple versions of stable-baselines3[extra] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of stable-baselines3[extra] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort

In [2]:
import gym
from stable_baselines3 import DQN
from stable_baselines3.dqn.policies import CnnPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from gym.wrappers.record_video import RecordVideo
import os
import shutil
from google.colab import drive

# **步驟 1: 掛載 Google 雲端硬碟**
drive.mount('/content/drive', force_remount=True)

# **步驟 2: 設定影片存放的本地與雲端路徑**
video_folder = './videos'  # 本地錄製影片資料夾
cloud_folder = '/content/drive/My Drive/videos'  # Google 雲端硬碟目標資料夾

# 確保影片資料夾不存在，避免舊檔覆蓋
if os.path.exists(video_folder):
    shutil.rmtree(video_folder)
os.makedirs(video_folder, exist_ok=True)

# 確保雲端資料夾存在
os.makedirs(cloud_folder, exist_ok=True)

# **步驟 3: 創建環境並設置參數**
env = gym.make('BreakoutNoFrameskip-v4', render_mode='rgb_array')
env.metadata['render_fps'] = 30  # 手動設置幀率

# 使用 DummyVecEnv 包裝環境
vec_env = DummyVecEnv([lambda: env])

# **步驟 4: 創建 DQN 模型**
model = DQN(
    policy=CnnPolicy,
    env=vec_env,
    learning_rate=1e-4,
    buffer_size=1_000_000,
    learning_starts=50_000,
    batch_size=32,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    target_update_interval=10_000,
    exploration_fraction=0.1,
    exploration_final_eps=0.01,
    verbose=1
)

# **步驟 5: 開始訓練模型**
model.learn(total_timesteps=2*10**5)

# **步驟 6: 保存模型**
model.save("dqn_breakout")

# **步驟 7: 測試模型並錄製影片**
test_env = gym.make('BreakoutNoFrameskip-v4', render_mode='rgb_array')
test_env.metadata['render_fps'] = 30
test_env = RecordVideo(test_env, video_folder=video_folder, episode_trigger=lambda x: True)

# 重置環境並解包返回值
reset_result = test_env.reset()
if isinstance(reset_result, tuple):  # 新版本 Gym 返回 (obs, info)
    obs, info = reset_result
else:  # 舊版本只返回 obs
    obs, info = reset_result, {}

while True:
    # 模型預測動作
    action, _states = model.predict(obs)

    # 適配不同版本的 step 返回值
    step_result = test_env.step(action)
    if len(step_result) == 5:  # 包含 truncated
        obs, rewards, dones, truncated, info = step_result
        dones = dones or truncated
    elif len(step_result) == 4:  # 標準返回格式
        obs, rewards, dones, info = step_result
    else:
        raise ValueError(f"Unexpected step return format from environment: {step_result}")

    # 判斷遊戲是否結束
    if dones:
        break

test_env.close()

# **步驟 8: 將影片移動到 Google 雲端硬碟**
video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith('.mp4')]

if len(video_files) > 0:
    for video_file in video_files:
        destination = os.path.join(cloud_folder, os.path.basename(video_file))
        shutil.move(video_file, destination)
        print(f"影片已移動到雲端硬碟：{destination}")
else:
    print("未找到影片文件，請檢查錄製過程是否正常。")

# **步驟 9: 列出雲端硬碟中的影片**
uploaded_videos = [os.path.join(cloud_folder, f) for f in os.listdir(cloud_folder) if f.endswith('.mp4')]
print("雲端硬碟中的影片文件：", uploaded_videos)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 201.62GB > 52.40GB
  warnings.warn(


Using cuda device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.874    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1629     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2538     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.738    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1686     |
|    time_elapsed     | 3        |
|    total_timesteps  | 5299     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.605    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 1708     |
|    time_elapsed     | 4        |
|    total_timesteps  | 7984     |
----------------------------------
----------------------------------
| rollout/          

/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
影片已移動到雲端硬碟：/content/drive/My Drive/videos/rl-video-episode-0.mp4
雲端硬碟中的影片文件： ['/content/drive/My Drive/videos/rl-video-episode-0.mp4']
